In [1]:
import rsa
import json
import secrets
import random
import string
import hashlib
from Crypto.Cipher import AES
import json
from base64 import b64encode , b64decode
from Crypto.Util.Padding import pad , unpad
from Crypto.Random import get_random_bytes
import hmac

**PUBLIC & PRIVATE KEY GENERATION**

In [2]:
"""
The code defines a function called "generate_pb_pr_keys" that generates a pair of RSA public and private keys with a key size of 1024 bits. 
These keys are then saved as separate files in a specific directory, with the file names based on the "pb_pr_author" parameter.
"""
def generate_pb_pr_keys(pb_pr_author):
    try:
        pb , pr = rsa.newkeys(1024)
        pb_file_name = "./database/public_private_key_pairs/"+pb_pr_author+"_public_key"
        pr_file_name = "./database/public_private_key_pairs/"+pb_pr_author+"_private_key"
        with open(pb_file_name,"wb") as f:
            f.write(pb.save_pkcs1(format="PEM"))
        with open(pr_file_name,"wb") as f:
            f.write(pr.save_pkcs1(format="PEM"))
    except Exception as e:
        print(e)


"""
This code defines a function called "get_public_key" that takes a parameter "public_key_author". 
It attempts to load a public key file from a specified directory using the author's name and the suffix "_public_key". 
If successful, it returns the loaded public key; otherwise, it prints any exception that occurred during the process.
"""
def get_public_key(public_key_author):
    try:
        pb_file_name = "./database/public_private_key_pairs/"+public_key_author+"_public_key"
        with open(pb_file_name,"rb") as f:
            pb = rsa.PublicKey.load_pkcs1(f.read())
        return pb
    except Exception as e:
        print(e)

"""
This code defines a function called "get_private_key" that takes a parameter "private_key_author". 
It attempts to load a private key file from a specified directory using the author's name and the suffix "_private_key". 
If successful, it returns the loaded private key; otherwise, it prints any exception that occurred during the process.
"""
def get_private_key(private_key_author):
    try:
        pr_file_name = "./database/public_private_key_pairs/"+private_key_author+"_private_key"
        with open(pr_file_name,"rb") as f:
            pr = rsa.PrivateKey.load_pkcs1(f.read())
        return pr
    except Exception as e:
        print(e)

**SYMETRIC KEY GENERATION**

In [3]:
"""
This code defines a function called "generate_symmetric_key" that takes a parameter "symmetric_key_author".
It generates a 32-byte (256-bit) symmetric key using the "secrets" module.
It then writes the generated key to a file in a specified directory using the author's name and the suffix "_symmetric_key.txt".
"""
def generate_symetric_key(symetric_key_author: str):
    symetrickey_file_name = "./database/symetric_keys/" + \
        symetric_key_author+"_symetric_key.txt"
    # Generate a 32-byte (256-bit) symetric key
    key = secrets.token_bytes(32)
    with open(symetrickey_file_name, "wb") as key_file:
        key_file.write(key)

"""
This code defines a function called "get_symmetric_key" that takes a parameter "symmetric_key_author". 
It attempts to read a symmetric key from a file in a specified directory using the author's name and the suffix "_symmetric_key.txt". 
If successful, it returns the read key as a string; otherwise, it prints any exception that occurred during the process and returns the string "None".
"""
def get_symetric_key(symetric_key_author: str):
    symetrickey_file_name = "./database/symetric_keys/" + \
        symetric_key_author+"_symetric_key.txt"
    key = ""
    try:
        with open(symetrickey_file_name, "rb") as key_file:
            key = key_file.read()
    except Exception as e:
        print(e)
        key = "None"
    return key

**RSA ENCRYPTION & DECRYPTION**

In [4]:
"""
This code defines a function called "encrypt_with_rsa_public" that takes two parameters: "message" and "author_name". 
It first checks if the message is already in bytes format, otherwise it encodes it as bytes. 
It then tries to retrieve the public key associated with the specified "author_name" using the "get_public_key" function. 
If successful, it encrypts the message using the "RSA" encryption algorithm with the obtained public key and returns the encrypted message. 
Any exceptions that occur during the process are printed.
"""
def encrpyt_with_rsa_public(message, author_name:str):
    if isinstance(message,bytes):
        message= message
    else:
        message = message.encode()
    try:
        pb = get_public_key(author_name)
        encrypted_message = rsa.encrypt(message,pb)
        return encrypted_message
    except Exception as e:
        print(e) 

In [5]:
"""
This code defines a function called "decrypt_with_rsa_private" that takes two parameters: "encrypted_message" and "author_name". 
It attempts to retrieve the private key associated with the specified "author_name" using the "get_private_key" function. 
If successful, it decrypts the "encrypted_message" using the "RSA" decryption algorithm with the obtained private key and returns the decrypted message. 
Any exceptions that occur during the process are printed.
"""
def decrypt_with_rsa_private(encrypted_message , author_name:str):
    try:
        pr = get_private_key(author_name)
        decrypted_message = rsa.decrypt(encrypted_message,pr)
        return decrypted_message
    except Exception as e:
        print(e) 

**RANDOM MESSAGE GENERATOR**

In [6]:
"""
This code defines a function called "generate_random_text" that takes a parameter "length".
It generates a random string of the specified length by randomly choosing characters from the set of ASCII letters (both lowercase and uppercase). 
The function then returns the generated random text.
"""
def generate_random_text(length):
    letters = string.ascii_letters
    text = ''.join(random.choice(letters) for _ in range(length))
    return text

**SHA256 HASH ALGORITHM for 𝐻(𝑚)**

In [7]:
"""
This code defines a function called "get_hash_message_digest" that takes a parameter "message". 
It computes the "SHA-256 hash digest" of the parameter "message" encoded in "UTF-8". 
It then prints the hash value in hexadecimal format and returns the computed digest.
"""
def get_hash_message_digest(message):
    digest = rsa.compute_hash(message.encode('utf-8'), 'SHA-256')
    print("Message digest hash value in hex:",digest.hex())
    return digest

**DIGITAL SIGNATURE(i.e. encryption with 𝐾𝐴−) & VERIFICATION**

In [8]:
"""
This code defines a function called "get_digital_signature" that takes two parameters: "message" and "private_key_author". 
It attempts to load the private key associated with the specified "private_key_author" from a file. 
If successful, it computes the "hash digest" of the message using the "get_hash_message_digest" function, and then generates a "digital signature" 
for the hash using the loaded private key and the "SHA-256 "algorithm. 
The function returns the generated signature. Any exceptions that occur during the process are printed.
"""
def get_digital_signature(message,private_key_author):
    try:
        pr_file_name = "./database/public_private_key_pairs/"+private_key_author+"_private_key"
        with open(pr_file_name,"rb") as f:
            pr = rsa.PrivateKey.load_pkcs1(f.read())

        hash = get_hash_message_digest(message)
        signature = rsa.sign_hash(hash, pr, 'SHA-256')
        return signature
    except Exception as e:
        print(e) 

In [9]:
"""
This code defines a function called "verify_digital_signature" that takes three parameters: "message", "signature", and "author_name". 
It attempts to load the public key associated with the specified "author_name" from a file. 
If successful, it "verifies the digital signature" by comparing it against the provided message using the loaded public key and the "SHA-256 algorithm".
If the verification succeeds, it prints "Verified" and returns True. 
If the verification fails with a rsa.pkcs1.VerificationError, it prints "Unconfirmed" and returns False. 
Any other exceptions that occur during the process are printed and it also returns False.
"""
def verify_digital_signature(message,signature,author_name):
    try:
        pb_file_name = "./database/public_private_key_pairs/"+author_name+"_public_key"
        with open(pb_file_name,"rb") as f:
            pb = rsa.PublicKey.load_pkcs1(f.read())
        res = rsa.verify(message.encode('utf-8'), signature, pb)
        if res == "SHA-256":
            print("Verified")
            return True
    except Exception as e:
        if isinstance(e,rsa.pkcs1.VerificationError):
            print("Unconfirmed")
            return False
        else:
            print(e)
            return False

**AES CBC ENCRYPTION & DECRYPTION**

In [10]:
"""
This code defines a function called "aes_cbc_encryption" that takes two parameters: "message" and "symmetric_key_author". 
It first encodes the message as bytes using UTF-8 encoding. 
It then retrieves the symmetric key associated with the specified "symmetric_key_author" using the "get_symmetric_key" function. 
Using the key, it initializes an "AES cipher in Cipher Block Chaining (CBC)" mode. 
The message bytes are padded to the appropriate block size, encrypted using the cipher, and encoded in Base64. 
The initialization vector (IV) and ciphertext are also Base64 encoded. 
Finally, the result is serialized as a JSON object and returned.
"""
def aes_cbc_encryption(message,symetric_key_author):
    message_in_bytes = message.encode("utf-8")
    key = get_symetric_key(symetric_key_author)
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message_in_bytes, AES.block_size))
    key = b64encode(key).decode('utf-8')
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    result = json.dumps({'iv':iv, 'ciphertext':ct})
    return result


"""
This code defines a function called "aes_cbc_decryption" that takes two parameters: "aes_encrypted_message" and "symmetric_key_author". 
It attempts to "decrypt an AES-CBC encrypted message". It first loads the Base64-encoded data from the aes_encrypted_message JSON object. 
The initialization vector (IV) and ciphertext are decoded from Base64. 
The symmetric key associated with the specified symmetric_key_author is retrieved using the get_symmetric_key function. 
An "AES cipher is initialized in CBC mode" with the key and IV. 
The ciphertext is decrypted, unpadded, and printed as the original message if successful. 
If there are any errors during decryption, such as a ValueError or KeyError, it prints "Incorrect decryption".
"""
def aes_cbc_decryption(aes_encrypted_message,symetric_key_author):
    try:
        b64 = json.loads(aes_encrypted_message)
        iv = b64decode(b64['iv'])
        ct = b64decode(b64['ciphertext'])
        key = get_symetric_key(symetric_key_author)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        pt = unpad(cipher.decrypt(ct), AES.block_size)
        print("The message was: ", pt.decode('utf-8'))
    except (ValueError, KeyError):
        print("Incorrect decryption")

**HMAC-SHA256 for MESSAGE AUTH CODE**

In [11]:
"""
This code defines a function called "generate_mac" that takes two parameters: message and "symmetric_key_author". 
It generates a "Message Authentication Code (MAC) using HMAC-SHA256". 
It retrieves the symmetric key associated with the specified "symmetric_key_author" using the "get_symmetric_key" function. 
The HMAC is computed by feeding the key and the message into the "hashlib library's hmac.new() function with SHA256 as the hashing algorithm". 
The resulting MAC digest is printed in hexadecimal format and returned as bytes.
"""
def generate_mac(message, symetric_key_author):
    hmac_sha256 = hmac.new(get_symetric_key(symetric_key_author), message, hashlib.sha256)
    print("Message Authentication Code (MAC) in Hex: ",hmac_sha256.digest().hex())
    return hmac_sha256.digest()

**PART 1: Application of Basic Cryptographic Functions**

>**1) Generation of public-private key pairs.**

In [12]:
author = "korçak"
generate_pb_pr_keys(author)
public_key = get_public_key(author)
private_key = get_private_key(author)
public_key , private_key

(PublicKey(129558541172063031598418691573335871332934994701055662120295220944176098068494426879113621423283721008567061860514906587384339323296705106741028670837646394440275085311371128623162692204857786283869159637203847469065141892963945284381775055404707793583107328094275787586645176653645420359990534888339511375867, 65537),
 PrivateKey(129558541172063031598418691573335871332934994701055662120295220944176098068494426879113621423283721008567061860514906587384339323296705106741028670837646394440275085311371128623162692204857786283869159637203847469065141892963945284381775055404707793583107328094275787586645176653645420359990534888339511375867, 65537, 48000533106472106905125413889744240533362750909201405800431638002129176453013003021983884550387598300334419168334871093401882037764427677446006314981970694650634259259178347484292564598167771994210963126787997079915413981659587509087571296561487753427216590675723456225913753828569457608855990658577632614881, 531953164485847247575569227065

>**2) Generation of Symmetric keys**

In [13]:
#determine the owner of public and private keys.
pub_priv_key_author = "korçak"

In [14]:
#determine the owner of symetric key.
symetric_key_author="taylan"

#generate a new symetic key and show.
generate_symetric_key(symetric_key_author)
symetric_key = get_symetric_key(symetric_key_author)
symetric_key

b'\xf9$\x10\xa0]\x83R\xe8!\x9cr\x0b^\xa0\x88H\xcd\xd2\x96\xd7\x83-\xfb\x1cg?zo:`\xa9\xee'

In [15]:
#encrypt the symetric key by using RSA algorithm and a public key which is needed for encryption.
encrypted_message= encrpyt_with_rsa_public(message=symetric_key,author_name=pub_priv_key_author)
encrypted_message

b"\x8c\x11\xf4\x9e17\\\x1b\xe6rsd\xed\xc0\xcb\x92\xe3\xd6I\x1c~UY\xd1\xbe\xee\xdd\x0cY\x1f\x08R\x94\x1a\x81\xa4W\x87\xf7\xf3\x16\xa3(\xa2\xf5\xa4@\xdf3\xd8\x86\xc3jN\x83\xaa'\x06D\xac\x9a\x87\xa6\xf6[\xb2\x99\xe4\x02c\x1e\x12\x1a>\xb4\x05\xef\x0c\x89\xbc\xaf\x97\xd5\x10h\x11\x03\xb6\xf1+\x81q\xc2VW\xeaEWU&\x8e\x85\x06\x10@b\xe7\x96T\xcd\xb2\x86\xd0\xaf\x17BU\x1d\xdam\xb1\xbb\xb1{S1l\xae"

Encrypted Message = 

b'=/\xd5[pacn\x8a\xb4\x9e\x96\x0f\xa5\x86\xcf\x9b8\xefS\x05\xf2\x159J\xb8\x1a2\x84\x9f\r\x9a(\x9e\xd6\x05\xc8.\xd1(\'\xfc^\x9e].\'\xd5{*\xb2\xd0\x94v\xe9\xd9\x7f|\x86\xb1UU\x11\xf8\x16\x06+O\xc9\xcd\xcb\xcb#\x88\x06\xef\xe5"\xf0\x9bV\xbf\xa3}y\xd2\x82\x01\xc8\xbb\xce\\\xa6f\xe1f\xb8\xab\x03\xa0\xd5q\xaf\rLz\x87O\x9cp\xfa\xba\t\x0c\xcbn*)\xf6\xba\x03@\xbc\xee\xa3$\x98\x82'

In [16]:
#decrypt the encrypted message by using RSA algorithm and a private key which is needed for encryption.
decrypted_message= decrypt_with_rsa_private(encrypted_message=encrypted_message,author_name=pub_priv_key_author)
decrypted_message

b'\xf9$\x10\xa0]\x83R\xe8!\x9cr\x0b^\xa0\x88H\xcd\xd2\x96\xd7\x83-\xfb\x1cg?zo:`\xa9\xee'

>>Message is same afer the encryption and decryption

>**3) Generation and Verification of Digital Signature**

In [17]:
#determine the owner of public and private keys.
pub_priv_key_author = "korçak"

#generate a random message with char lenght = 110.
txt = generate_random_text(length=110)
print("Message = ",txt)

Message =  PjKNbGMeaCLQyemgTQsuYwIKsqmtUJlGxiQUdtqXFXrxmYicYSWPMqlJzArXrJxAdtXYWixRLjgpgTVeMUiMfvkkLLMSjNdZQmVLIaFsTtDTBl


In [18]:
#apply digital signature to message and show.
digital_signature=get_digital_signature(message=txt,private_key_author=pub_priv_key_author)
digital_signature

Message digest hash value in hex: 1b3c37c69db7f0ea0a69ae93abc5bc4f00eb1f041e377703aa980f3e23c4bab8


b'\x9f\xfa\x89=\x15\x11\xc4\xe8\xcenN\x9ck\xbf\xda\xdf\x89\t\xab\xba\xe0\x0clk\xe0rK\xc4\xfc\x1a\xb7\x8f\x0fh\xd1\x8em1d\xfa\xdd\xd1\xebf\x11\xfa\xfe_S\xdf\xbc\xba\xf7c#=R\x90\x87\xd1a\xfbE\xfb&\x13\xbe>\x9c>\xa07\x8c\xccz\x16\x8a\x11#\xfe6\xf9\x8a\x1e\xd2)\xf9\xa3\x85\xeb\x9f1#\xf8\xeeo\x9b\x04!\xc2Dk\xb2}\xd5\xa8\xaf\xe0i\x93\x08i\xad\xe1v\\\xf6\x9f\xdf\x92p\x16\xaf\x87\xcfc|\xb8'

Digital Signature = 
b'zKK\x0c\xc4\xc9\xfb\x85S\xf9\xf1G\x03\xdd\xb5l^k\x9b\x8a\x117Q\xfaY\x14\xac\xf6a\xefQ\xe6\xb8\x90S%"\xe8\xae\x93\xd9\x92\x14\xd2\x1d\xa3\xbc\xc2oQ\nm\xf0\r:9\x07\x13R\x83\xc81\x97\xfc\xe4\x8c\xb6\x830]M\x048\'\x1e\xf5\xfaA^ @zy\x91\xc5\x0c\xde\x84Z\xd0\x12\x01\x93)c\xbf\xc4\x96\xe4\xdd4\x08\xf7\x99\xccM\x1bGs\x8cc_38\xe8\xc8\xe7\xd6\xbaeS/\xda%\x15M\xd9\xba'

In [19]:
#verifying the digital signature and it's results
verify_digital_signature(message=txt,signature=digital_signature,author_name=pub_priv_key_author)

Verified


True

>**4) AES Encryption/Descryption**

In [20]:
#determine the owner of symetric key.
symetric_key_author = "taylan"

#generate a random message with char lenght = 110.
txt = generate_random_text(length=110)
print("Message = ",txt)

Message =  bbHeUCfLoUaeXZRUSwOfDwwJkgvizzhabQprQXeuIiRgIpFbCORdwyNJMkSjYEzXcdAhvYzkXHDyvxDiuOzahVWADWUQmJOAUoMXBsuaOLdWhp


In [21]:
#encrypt the message with AES CBC algorithm. IV(Initialization Vector) and Ciphertext are shown in the results.
aes_encr = aes_cbc_encryption(message=txt,symetric_key_author=symetric_key_author)
aes_encr

'{"iv": "hx8MN4NYLGv3E/Ja+4+bGA==", "ciphertext": "GsFTsBQDQa2sfHKPH7mFWRxHnCP1Q6wQOTfWuvGD8eBPXRiQ4RC5d+y6OvK4pxkGav7bpPv/x+x5VGgbyybkCl0MrRVh9mlP+dnYL3R88KxK0TdTr5i/kY4bMcVR9+tGHaB+9+qJKuFHiQpXCGU/FA=="}'

In [22]:
#decrypt the message with AES CBC algorithm. Message is
aes_decr = aes_cbc_decryption(aes_encrypted_message=aes_encr,symetric_key_author=symetric_key_author)
aes_decr

The message was:  bbHeUCfLoUaeXZRUSwOfDwwJkgvizzhabQprQXeuIiRgIpFbCORdwyNJMkSjYEzXcdAhvYzkXHDyvxDiuOzahVWADWUQmJOAUoMXBsuaOLdWhp


>>Since message is not changes. It successfuly encrypted and decrypted.

>**5) Message Authentication Codes**

In [23]:
example_message=b"mymessage"
symetric_key_author = "taylan"

In [24]:
hmac = generate_mac(message=example_message,symetric_key_author=symetric_key_author)
hmac

Message Authentication Code (MAC) in Hex:  94a55debc9ee54b1aeca42511f28d3125bf1ce8e55ffe455933e560b3a983461


b'\x94\xa5]\xeb\xc9\xeeT\xb1\xae\xcaBQ\x1f(\xd3\x12[\xf1\xce\x8eU\xff\xe4U\x93>V\x0b:\x984a'